# Introduction

This is an [R Markdown](http://rmarkdown.rstudio.com) Notebook for the analysis of the [bird ringing data Netherlands 1960-1990 part 1, led by  Henk van der Jeugd](https://doi.org/10.17026/dans-2ch-6s6r).

In an R notebook we can combine text, code and data together. The text is formated using [Markdown](), whereas data and code are located within ` ```{r}``` `blocks.

An R Notebook can rely on external libraries. The following block adds the required `knitr` library, as well as some additional ones for data wrangling the calculation of indices.

In [1]:
# Data Analysis Libraries
library(dplyr)
library(tidyr)

# [Community Ecology Package](https://cran.r-project.org/web/packages/vegan/index.html)
library(vegan)

# Visualization Libraries
library(ggplot2)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: permute
Loading required package: lattice
This is vegan 2.4-4


# Loading and Cleaning Data

For our first step, we will load the data and then view the top records as well as a summary of all variables included.

In [2]:
dansDataSet <- read.csv(file = "Export_DANS_Parels_van_Datasets_Vogeltrekstation.csv", header = TRUE)

head(dansDataSet)
summary(dansDataSet)

CatchID,Remarks_Public,SchemeCode,Ringnumber,Ringtype,RingID,SpeciesCode,CatchDate,CatchTime,MetalRingInformation,...,Location,Code,Own.location.description,Latitude,Longitude,Accuracy.of.coordinates,Catching.Method,Catching.Lures,Condition,Circumstances
80002,Import historische Data 2a,NLA,....270376,8.0,6270108,5900,1962-07-01 00:00:00.000,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NULL,53.1017,4.767595,11,Z,U,8,20
80003,Import historische Data 2a,NLA,....270377,8.0,6270109,5900,1962-07-01 00:00:00.000,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NULL,53.1017,4.767595,11,Z,U,8,20
80004,Import historische Data 2a,NLA,....270378,8.0,6270110,5900,1962-07-01 00:00:00.000,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NULL,53.1017,4.767595,11,Z,U,8,20
80005,Import historische Data 2a,NLA,....270379,8.0,6270111,5900,1962-07-01 00:00:00.000,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NULL,53.1017,4.767595,11,Z,U,8,20
80006,Import historische Data 2a,NLA,....270380,8.0,6270112,4500,1962-07-01 00:00:00.000,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NULL,53.1017,4.767595,11,Z,U,8,20
80007,Import historische Data 2a,NLA,....270381,8.0,6270113,5900,1962-07-02 00:00:00.000,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NULL,53.1017,4.767595,11,Z,U,8,20


    CatchID                            Remarks_Public  SchemeCode 
 Min.   :   80002   Import Historische Data 2 :39391   NLA:66352  
 1st Qu.:  257426   Import historische Data 2a:26962   NLL:    1  
 Median :10419291                                                 
 Mean   : 6807335                                                 
 3rd Qu.:10435879                                                 
 Max.   :10452467                                                 
                                                                  
      Ringnumber       Ringtype         RingID          SpeciesCode   
 ...3365401:    2   NULL   :60487   Min.   :   15930   Min.   :    0  
 ....187951:    1   2.5    : 4143   1st Qu.: 6433496   1st Qu.: 4930  
 ....187952:    1   5.5    :  631   Median :10339764   Median :10990  
 ....187953:    1   11.0   :  529   Mean   : 8450059   Mean   :10747  
 ....234500:    1   9.0    :  491   3rd Qu.:10356351   3rd Qu.:15980  
 ....262416:    1   10.0   :   45   Ma

We observe that even though the data was loaded correctly, they are not used in the best possible way. For example, `Ringnumber`, `CatchDate` and `Age` are used as words rather than as numeric values. Also, missing values are defined as `NULL` which is not recognized as such by R (the correct value would be `NA`). The next block tidies the data, so that that each attribute is treated as originally intended.

In [3]:
dansDataSet <- data.frame(lapply(dansDataSet, function(x) { gsub("NULL", NA, x) }))

dansDataSet$Ringnumber <- as.numeric(dansDataSet$Ringnumber)
dansDataSet$CatchDate <- as.Date(dansDataSet$CatchDate)
dansDataSet$Age <- as.numeric(dansDataSet$Age)
dansDataSet$Broodsize <- as.numeric(dansDataSet$Broodsize)
dansDataSet$PullusAge <- as.numeric(dansDataSet$PullusAge)
dansDataSet$CatchDate <- as.Date(dansDataSet$CatchDate)

head(dansDataSet)
summary(dansDataSet)

CatchID,Remarks_Public,SchemeCode,Ringnumber,Ringtype,RingID,SpeciesCode,CatchDate,CatchTime,MetalRingInformation,...,Location,Code,Own.location.description,Latitude,Longitude,Accuracy.of.coordinates,Catching.Method,Catching.Lures,Condition,Circumstances
80002,Import historische Data 2a,NLA,24,8.0,6270108,5900,1962-07-01,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NA,53.1016990666569,4.7675945487984,11,Z,U,8,20
80003,Import historische Data 2a,NLA,25,8.0,6270109,5900,1962-07-01,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NA,53.1016990666569,4.7675945487984,11,Z,U,8,20
80004,Import historische Data 2a,NLA,26,8.0,6270110,5900,1962-07-01,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NA,53.1016990666569,4.7675945487984,11,Z,U,8,20
80005,Import historische Data 2a,NLA,27,8.0,6270111,5900,1962-07-01,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NA,53.1016990666569,4.7675945487984,11,Z,U,8,20
80006,Import historische Data 2a,NLA,28,8.0,6270112,4500,1962-07-01,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NA,53.1016990666569,4.7675945487984,11,Z,U,8,20
80007,Import historische Data 2a,NLA,29,8.0,6270113,5900,1962-07-02,----,2,...,"Stappeland 48, Texel, De Koog, Noord-Holland, NL",NL00,NA,53.1016990666569,4.7675945487984,11,Z,U,8,20


     CatchID                         Remarks_Public  SchemeCode 
 10413077:    1   Import Historische Data 2 :39391   NLA:66352  
 10413078:    1   Import historische Data 2a:26962   NLL:    1  
 10413079:    1                                                 
 10413080:    1                                                 
 10413081:    1                                                 
 10413082:    1                                                 
 (Other) :66347                                                 
   Ringnumber       Ringtype          RingID       SpeciesCode   
 Min.   :    1   2.5    : 4143   10346836:    2   4930   :10256  
 1st Qu.:16589   5.5    :  631   10045017:    1   14640  : 4842  
 Median :33176   11.0   :  529   1016867 :    1   16360  : 4146  
 Mean   :33176   9.0    :  491   10338352:    1   16620  : 3971  
 3rd Qu.:49764   10.0   :   45   10338353:    1   9920   : 3691  
 Max.   :66352   (Other):   27   10338354:    1   4850   : 3070  
                 N

We can see that the data is much more better formatted and useful for further analysis.

# Subsetting our data

Let's now create a few subsets of the original data. Subset #1 `dansDataSet_Castricum` will contain all the unique records for which `Location` is `Castricum, Noord-Holland, NL`. Then we will group the records by species and catch date, and calculate number of each species in the particular catch date.

In [4]:
dansDataSet_Castricum <- dansDataSet %>%
  filter(Location == "Castricum, Noord-Holland, NL") %>%
  select(unique.RingID = RingID, Species, CatchDate) %>%
  group_by(Species, CatchDate) %>%
  summarise(count = n())

We could further filter this subset for a particular species. For example, the code below will retrieve all unique observations of Northern Lapwing in Castricum, Noord-Holland, NL.

In [5]:
dansDataSet_lapwing <- dansDataSet %>%
  filter(Location == "Castricum, Noord-Holland, NL") %>%
  select(unique.RingID = RingID, Species, CatchDate) %>%
  group_by(Species, CatchDate) %>%
  filter(as.POSIXct(CatchDate) >= as.POSIXct("1970-01-01 00:00:01")) %>%
  filter(Species == "Northern Lapwing") %>%
  summarise(count = n())

Our second subset will create a matrix of the distribution of unique species across the different locations. This will consequently allow us to calculate some diversity indexes.

In [6]:
dansDataSet_distribution <- dansDataSet %>%
  select(unique.RingID = RingID, Species, Location) %>%
  group_by(Species, Location) %>%
  summarise(count = n()) %>%
  filter(count > 0) %>%
  na.omit()

# spread(data, key, value)
#   data: A data frame
#   key: The (unquoted) name of the column whose values will be used as column headings.
#   value:The (unquoted) names of the column whose values will populate the cells
dansDataSet_distribution_matrix <- dansDataSet_distribution %>%
  spread(Location, count)

We can also create a more specific subset, i.e. of species that have at least 100 unique observations in a given location. This will allow for a cleaner figure.

In [7]:
dansDataSet_distribution_min100 <- dansDataSet %>%
  select(unique.RingID = RingID, Species, Location) %>%
  group_by(Species, Location) %>%
  summarise(count = n()) %>%
  filter(count > 100) %>%
  na.omit()

# Using the `vegan` package

We will now use the [`vegan`](https://cran.r-project.org/web/packages/vegan/index.html) package to calculate the diversity in the locations.

## Transforming the data to `vegan` requirements

In [8]:
dansDataSet_distribution_zero <- dansDataSet_distribution_matrix
dansDataSet_distribution_zero[is.na(dansDataSet_distribution_zero)] <- 0
dansDataSet_distribution_zero <- t(dansDataSet_distribution_zero[,2:length(dansDataSet_distribution_zero)])

## Calculating diversity: **Shannon**, **Simpson** and **Inverted Simpson**.

For each of these indexes, we are going to call the corresponding function from vegan, using the default parameters:

Shannon or Shannon–Weaver (or Shannon–Wiener) index is defined as:

$$H = -\sum_{n=1}^{R} p_i ln_b(p_i) = 1$$

where $p_i$ is the proportional abundance of species $i$ and $b$ is the base of the logarithm. It is most popular to use natural logarithms, but some argue for base $b = 2$.

Both variants of Simpson's index are based on $D = \sum_{n=1}^{R}p_i^2$. Choice simpson returns $1-D$ and invsimpson returns $\frac{1}{D}$.

In [9]:
Hshannon <- diversity(dansDataSet_distribution_zero, index = "shannon", MARGIN = 1, base = exp(1))
simp <- diversity(dansDataSet_distribution_zero, "simpson", MARGIN = 1)
invsimp <- diversity(dansDataSet_distribution_zero, "inv", MARGIN = 1)

## Calculating species richness

The function `rarefy` gives the expected species richness in random subsamples of size sample from the community. The size of sample should be smaller than total community size, but the function will silently work for larger sample as well and return non-rarefied species richness (and standard error equal to 0). If sample is a vector, `rarefaction` is performed for each sample size separately. Rarefaction can be performed only with genuine counts of individuals. The function rarefy is based on Hurlbert's (1971) formulation, and the standard errors on Heck et al. (1975).

In [10]:
r.2 <- rarefy(dansDataSet_distribution_zero, 2)

Warning message in rarefy(dansDataSet_distribution_zero, 2):
"Requested 'sample' was larger than smallest site maximum (1)"

## Calculating `fisher.alpha`

This function estimates the $a$ parameter of Fisher's logarithmic series. The estimation is possible only for genuine counts of individuals. The function can optionally return standard errors of $a$. These should be regarded only as rough indicators of the accuracy: the confidence limits of $a$ are strongly non-symmetric and the standard errors cannot be used in Normal inference.

In [11]:
alpha <- fisher.alpha(dansDataSet_distribution_zero)

## Richness and Evenness

Species **richness** (S) is calculated by `specnumber` which finds the number of species. If MARGIN is set to 2, it finds frequencies of species. **Pielou's evenness** (J) is calculated by $\frac{H_shannon}{log(S)}$.

In [12]:
S <- specnumber(dansDataSet_distribution_zero, MARGIN = 1) ## rowSums(BCI > 0) does the same...
J <- Hshannon/log(S)

In order to have all these indeces together, we will put them in a single data frame as follows:

In [13]:
metrics <- data.frame(
  H_Shannon = Hshannon,
  H_Simp = simp,
  H_Inv_Simp = invsimp,
  rarefy = r.2,
  a = alpha,
  richness = S,
  evenness = J
)

# Results

Finally, let's also create some plots. First of all, let's create a plot based on our first subset, showing for each species and capture dates, the average age of the species captured.


In [14]:
png("subset1a1.png", width = 4000, height = 2000, res = 300, pointsize = 5)
ggplot(data=dansDataSet_Castricum, aes(x=CatchDate, y=Species, color=count)) +
  geom_point(aes(size=count))
dev.off()

png 
  2

![_First subset plot: points_](subset1a1.png)

In [15]:
png("subset1a2.png", width = 4000, height = 2000, res = 300, pointsize = 5)
ggplot(data=dansDataSet_Castricum, aes(x=CatchDate, y=count, colour=Species)) +
  geom_line()
dev.off()

png 
  2

![_First subset plot: lines_](subset1a2.png)

We can do the same plots for the single species that we looked into earlier (Northern Lapwing in Castricum, Noord-Holland, NL).

In [16]:
png("subset1b1.png", width = 4000, height = 2000, res = 300, pointsize = 5)
ggplot(data=dansDataSet_lapwing, aes(x=CatchDate, y=Species, color=count)) +
  geom_point(aes(size=count))
dev.off()

png 
  2

![_First subset plot: Northern Lapwin points_](subset1b1.png)

This is not really easy to interpret. However, we can now have a more interesting plot with the `lines` command, including a smoothing curve to show the overall trend:

In [17]:
png("subset1b2.png", width = 4000, height = 2000, res = 300, pointsize = 5)
ggplot(data=dansDataSet_lapwing, aes(x=CatchDate, y=count, colour=Species)) +
  geom_point(aes(x = CatchDate, y = count, colour = Species), size = 3) +
  stat_smooth(aes(x = CatchDate, y = count), method = "lm", formula = y ~ poly(x, 3), se = FALSE)
dev.off()

png 
  2

![_First subset plot: Northern Lapwin lines](subset1b2.png)

We can also create a plot based on the second subset. In this case, let's see how the distribution of species across the seven locations looks like.

In [18]:
lvls <- unique(as.vector(dansDataSet_distribution$Location))
png("subset2a.png", width = 4000, height = 2000, res = 300, pointsize = 5)
ggplot(data=dansDataSet_distribution, aes(x=Species, y=Location, color=Species)) + 
  geom_point(aes(size=count)) +
  theme(text=element_text(family="Arial", size=12*(81/169)),
        axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.3)) +
    scale_y_discrete(breaks=lvls[seq(1,length(lvls),by=10)]) #scale_y_discrete(labels = abbreviate)
dev.off()

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font fa

"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in gr

"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in gr

"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in gr

"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in gr

png 
  2

![_Second subset plot_](subset2a.png)

This is a very "dense" figure, so let's use the filtered version to see the most highly populated species.

In [19]:
png("subset2b.png", width = 4000, height = 2000, res = 300, pointsize = 5)
ggplot(data=dansDataSet_distribution_min100, aes(x=Species, y=Location, color=Species)) + 
  geom_point(aes(size=count)) +
  theme(text=element_text(family="Arial", size=12*(81/169)),
        axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.3))
dev.off()

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font family not found in Windows font database"Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
"font fa

png 
  2

![_Second subset plot_](subset2b.png)

Finally, let's have a figure showing all 5 indexes together.

In [20]:
png("metrics.png", width = 4000, height = 2000, res = 300, pointsize = 5)
plot(metrics, pch="+", col="blue")
dev.off()

png 
  2

![_Second subset plot_](metrics.png)

We could also show the most diverse sites (i.e. richness index over 10).

In [21]:
top10_site_metrics <- metrics %>%
  tibble::rownames_to_column() %>%
  filter(richness >= 10) %>%
  arrange(desc(richness))

top10_site_metrics

rowname,H_Shannon,H_Simp,H_Inv_Simp,rarefy,a,richness,evenness
"Castricum, Noord-Holland, NL",2.804250,0.9064043,10.684250,1.906937,7.561696,41,0.7551355
"Lageweideviaduct, Utrecht, West, Utrecht, NL",2.681204,0.9175863,12.133909,1.926325,7.853568,21,0.8806650
"Nieuwe Parklaan, Den Haag, Zuid-Holland, NL",2.565374,0.8940430,9.437788,1.908234,9.132867,19,0.8712606
"Terschelling, Terschelling Oosterend, Friesland, NL",2.039750,0.7846147,4.642843,1.789845,5.326671,18,0.7057049
"Vlieland, Friesland, NL",1.557635,0.6514916,2.869371,1.653678,4.208175,18,0.5389046
"Liebergerweg 301, Hilversum, Noord-Holland, NL",2.090347,0.8247599,5.706458,1.832468,5.191263,16,0.7539332
"Bergen, Egmond aan Zee, Noord-Holland, NL",1.780973,0.6983471,3.315068,1.714588,6.227126,13,0.6943502
"Wassenaar, Zuid-Holland, NL",1.275899,0.5731265,2.342615,1.576420,2.919873,12,0.5134595
"Bloemendaal, Noord-Holland, NL",1.889995,0.7935528,4.843854,1.808526,4.175936,11,0.7881890
"Koningin Wilhelminaweg 337, Groenekan, Utrecht, NL",1.788852,0.7817288,4.581456,1.787651,2.845495,11,0.7460090


# Conclusions

Jupyter notebooks and R is awesome!